## A python stream generating application

In [1]:
import time
import csv
import socket
from multiprocessing import Process
import random, string
import json

with open('./ClimateData-Part2.csv','r') as c2, open ('./FireData-Part2.csv','r') as f2:
    climate_reader = csv.reader(c2)
    fire_reader  = csv.reader(f2)
    climate = []
    fire  = []
    for row in climate_reader:
        climate.append(','.join(e.strip() for e in row) + '\n ')
    for line in fire_reader:
        line.append(''.join(random.choices(string.ascii_letters + string.digits, k=12)))
        fire.append(','.join(e.strip() for e in line) + '\n')
climate =  climate[1:]
fire = fire[1:]  

def generator(dataset):
    for e in dataset:
        yield e
climate_genrator = generator(climate)
fire_genrator = generator(fire)
def generate_stream(dataset,port):
    """
    The method that generate a stream sending to socket of the computer from a given dataset
    -------
    Parameters:
    dataset ---the generator object of the dataset
    port --- the usable port that the data stream can be sent through
    """ 
    host = 'localhost'

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    s.bind((host, port))
    s.listen(1)
    i = 0
    
    while True:
        try:
            conn, addr = s.accept()
            print("exit({})".format(i))
            i += 1
            while True:
                if dataset == climate_genrator:
                    try:#if the dataset is climate, we send one record and then pause 1 second
                        conn.send(next(dataset).encode())
                        time.sleep(1)
                    except StopIteration:
                        return
                elif dataset == fire_genrator:
                    try:#if the dataset is fire, we send five records and then pause 1 second
                        for i in range(5):
                            conn.send(next(dataset).encode())
                        time.sleep(1)
                    except StopIteration:
                        return                    
        except socket.error: 
            pass
        finally:
            conn.close()
            s.close()
    conn.close()
    s.close()
    return
                

if __name__ == '__main__':
    #here we must open the two sockets using mp so that it will be openn at the same time
    #otherwise the other port cannot be connected to pyspark DStream
    p1 = Process(target=generate_stream, args=(climate_genrator,9999))
    p2 = Process(target=generate_stream, args=(fire_genrator,8080))
    p1.start()
    p2.start()

exit(0)
exit(0)


Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zby0902/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zby0902/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zby0902/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zby0902/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-fc157d1482f2>", line 62, in generate_stream
    pass
  File "<ipython-input-1-fc157d1482f2>", line 62, in generate_stream
    pass
KeyboardInterrupt
KeyboardInterrupt
